In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import datasets, layers, models


In [2]:
train_data_dir = 'chest_xray/train/'
val_data_dir = 'chest_xray/val/'

In [3]:
val_generator = ImageDataGenerator().flow_from_directory(
        val_data_dir, 
        target_size=(64, 64), batch_size=16)

# Get all the data in the directory data/train (790 images), and reshape them
train_generator = ImageDataGenerator().flow_from_directory(
        train_data_dir, 
        target_size=(64, 64), batch_size=5216)

# Create the datasets
train_images, train_labels = next(train_generator)
val_images, val_labels = next(val_generator)

Found 16 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.


In [4]:
print(np.shape(train_images))
print(np.shape(train_labels))
print(np.shape(val_images))
print(np.shape(val_labels))

(5216, 64, 64, 3)
(5216, 2)
(16, 64, 64, 3)
(16, 2)


In [5]:
train_img_unrow = train_images.reshape(5216, -1).T
np.shape(train_img_unrow)


(12288, 5216)

In [6]:
val_img_unrow = val_images.reshape(16, -1).T
np.shape(val_img_unrow)

(12288, 16)

In [7]:
train_generator.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [8]:
train_labels_final = train_labels.T[[1]]
np.shape(train_labels_final)

(1, 5216)

In [9]:
val_labels_final = val_labels.T[[1]]
np.shape(val_labels_final)

(1, 16)

In [10]:
train_img_final = train_img_unrow/255
val_img_final = val_img_unrow/255

type(val_img_unrow)

numpy.ndarray

In [11]:
train_img_final.shape

(12288, 5216)

# Baseline - Logistic Regression

In [12]:
b = 0

In [13]:
def init_w(n):
    w = np.zeros((n, 1))
    return w

In [14]:
w = init_w(64*64*3)

In [15]:
def propagation(w, b, x, y):
    l = x.shape[1]
    y_hat = 1/(1 + np.exp(- (np.dot(w.T, x) + b)))                                  
    cost = -(1/l) * np.sum(y * np.log(y_hat) + (1-y)* np.log(1 - y_hat))    
    dw = (1/l) * np.dot(x,(y_hat - y).T)
    db = (1/l) * np.sum(y_hat - y)
    return dw, db, cost

In [16]:
def optimization(w, b, x, y, num_iterations, learning_rate, print_cost = False):
    
    costs = []
    
    for i in range(num_iterations):
        dw, db, cost = propagation(w, b, x, y)    
        w = w - learning_rate*dw
        b = b - learning_rate*db
        
        # Record the costs and print them every 50 iterations
        if i % 50 == 0:
            costs.append(cost)
        if print_cost and i % 50 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    
    return w, b, costs

In [17]:
def prediction(w, b, x):
    l = x.shape[1]
    y_prediction = np.zeros((1, l))
    w = w.reshape(x.shape[0], 1)
    y_hat = 1/(1 + np.exp(- (np.dot(w.T, x) + b))) 
    p = y_hat
    
    for i in range(y_hat.shape[1]):
        if (y_hat[0,i] > 0.5): 
            y_prediction[0, i] = 1
        else:
            y_prediction[0, i] = 0
    return y_prediction

In [18]:
def model(x_train, y_train, x_test, y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):

    b = 0
    w = init_w(np.shape(x_train)[0]) 

    # Gradient descent (≈ 1 line of code)
    w, b, costs = optimization(w, b, x_train, y_train, num_iterations, learning_rate, print_cost)
    
    y_pred_test = prediction(w, b, x_test)
    y_pred_train = prediction(w, b, x_train)

    # Print train/test errors
    print('train accuracy: {} %'.format(100 - np.mean(np.abs(y_pred_train - y_train)) * 100))
    print('test accuracy: {} %'.format(100 - np.mean(np.abs(y_pred_test - y_test)) * 100))

    output = {'costs': costs, 
              'y_pred_test': y_pred_test,  
              'y_pred_train' : y_pred_train,  
              'w' : w, 
              'b' : b, 
              'learning_rate' : learning_rate, 
              'num_iterations': num_iterations}
    
    return output

In [19]:
output = model(train_img_final, train_labels_final, val_img_final, val_labels_final, 
               num_iterations=500, learning_rate=0.01, print_cost=False)

train accuracy: 95.0536809815951 %
test accuracy: 87.5 %


In [20]:
output = model(train_img_final, train_labels_final, val_img_final, val_labels_final, 
               num_iterations=300, learning_rate=0.01, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 50: 1.524936
Cost after iteration 100: 0.566416
Cost after iteration 150: 0.315296
Cost after iteration 200: 0.188630
Cost after iteration 250: 0.163194
train accuracy: 94.1909509202454 %
test accuracy: 87.5 %


In [21]:
output = model(train_img_final, train_labels_final, val_img_final, val_labels_final, 
               num_iterations=200, learning_rate=0.005, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 50: 0.497045
Cost after iteration 100: 0.326489
Cost after iteration 150: 0.242337
train accuracy: 92.4463190184049 %
test accuracy: 81.25 %


In [22]:
output = model(train_img_final, train_labels_final, val_img_final, val_labels_final, 
               num_iterations=200, learning_rate=0.01, print_cost=True)

Cost after iteration 0: 0.693147
Cost after iteration 50: 1.524936
Cost after iteration 100: 0.566416
Cost after iteration 150: 0.315296
train accuracy: 92.48466257668711 %
test accuracy: 87.5 %


# MLP

In [23]:
train_images, train_labels = next(train_generator)
val_images, val_labels = next(val_generator)

In [24]:
train_images_mlp = (train_images / 255).astype('float32')
val_images_mlp = (val_images / 255).astype('float32')

In [26]:
model_1 = models.Sequential()
model_1.add(layers.Conv2D(filters=32, 
                        kernel_size=(2,2),
                        strides=(1,1),
                        activation='relu',
                        padding = 'same',
                        input_shape=(64, 64, 3),
                        data_format = 'channels_last'))
model_1.add(layers.MaxPooling2D(pool_size=(2,2),
                     strides=2))
model_1.add(layers.Flatten())        
model_1.add(layers.Dense(128))
model_1.add(layers.Activation('relu'))
#model_1.add(layers.Dropout(0.25))
model_1.add(layers.Dense(2))
model_1.add(layers.Activation('sigmoid'))

In [27]:
model_1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 32)        416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4194432   
                                                                 
 activation (Activation)     (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                        

In [28]:
model_1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model_1.fit(train_images_mlp, train_labels, epochs=5, batch_size=50)

Epoch 1/5
105/105 [==============================] - 5s 43ms/step - loss: 0.3627 - accuracy: 0.8744
Epoch 2/5
105/105 [==============================] - 4s 42ms/step - loss: 0.1338 - accuracy: 0.9544
Epoch 3/5
105/105 [==============================] - 5s 43ms/step - loss: 0.0942 - accuracy: 0.9666
Epoch 4/5
105/105 [==============================] - 5s 43ms/step - loss: 0.0839 - accuracy: 0.9695
Epoch 5/5
105/105 [==============================] - 5s 43ms/step - loss: 0.0696 - accuracy: 0.9734


In [ ]:
test_loss, test_acc = model_1.evaluate(val_images, val_labels)

1/1 [==============================] - 0s 182ms/step - loss: 13.5447 - accuracy: 0.9375
